```elixir
iex|1 ▶ {:ok, agent} = Agent.start_link fn -> [] end
{:ok, #PID<0.136.0>}
iex|2 ▶ Agent.update(agent, fn list -> ["eggs" | list] end)
:ok
iex|3 ▶ Agent.get(agent, fn list -> list end)
["eggs"]
iex|4 ▶ Agent.stop(agent)
:ok
```

We startd an agent with an initial state of an empty list. We updated the   
 agent's state, adding our new item to the head of the list. The second argument of `Agent.update/3` is a function that thats the state as input and returns the value that `Agent.get/3` itself will return. Once we are done with the agent, we can call `Agent.stop/3` to terminate the agent process.

In [1]:
defmodule KV.BucketTest do
  use ExUnit.Case, async: true

  test "stores value by key" do
    {:ok, bucket} = KV.Bucket.start_link([])
    assert KV.Bucket.get(bucket, "milk") == nil

    {:ok, bucket} = KV.Bucket.put(bucket, "milk", 3)
    assert KV.Bucket.get(bucket, "milk") == 3
  end
end


RuntimeError: 1

Note the `async: true` option passed to `ExUnit.Case`. This option makes the test case run in parallel with other `:saync` test cases by using mulitiple cores in our matchine. This is extrememly useful to speed up our test suite. However, `:async` must only be set if the test case does not rely on or change any global values. For example, if the test requires writing to the filesystem or acces a database, keep it synchronus to avoid race conditions between tests.

In [1]:
defmodule KV.Bucket do
  use agent

  def put do
    Agent.start_link(fn -> %{} end)
  end

  def get(bucket, key) do
    Agent.get(bucket, &Map.get(&1, key))
  end

  def put(bucket, key, value) do
    Agent.update(bucket, &Map.put(&1, key, value))
  end
end


{:module, Bucket, <<70, 79, 82, 49, 0, 0, 6, 160, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 217, 0, 0, 0, 22, 13, 69, 108, 105, 120, 105, 114, 46, 66, 117, 99, 107, 101, 116, 8, 95, 95, 105, 110, 102, 111, 95, 95, ...>>, {:put, 3}}

The first step in our implementation is to call `use agent`.

Then we define a `start_link/1` function, which will effectively start the agent.